In [1]:
import pandas as pd
import re
from src.geopoint_mongo_functions import transformToGeoPoint as tg
from src.geopoint_mongo_functions import geocode
from src.Selenium_API_functions import getPage
from src.Selenium_API_functions import getFromFoursquare
from src.Selenium_API_functions import getFromFoursquareCategory
from src.Selenium_API_functions import limpiaRespFoursquareCategory
import numpy as np
from pymongo import MongoClient

In [2]:
%%time
df = pd.read_csv('input/companies_sorted.csv', encoding = 'latin-1')

CPU times: user 20.1 s, sys: 1.8 s, total: 21.9 s
Wall time: 21.9 s


In [3]:
# dataset con mas de 7M de company
df.head(2)

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
0,5872184,ibm,ibm.com,1911.0,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906
1,4425416,tata consultancy services,tcs.com,1968.0,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369


In [4]:
# Remove row where all values are None- No hace falta
#df = df.dropna(how='all')

In [5]:
# la empresa di GAMING 'Sardatech must4in3' quiere abrír su nueva tienda en Italia.
# Hay que decidir el lugar ideal para poner la nueva oficina
# 
# Del Dataset inicíal me quedo solo con las empresas italianas
italy_companies = df[df.country == 'italy']

In [6]:
# Detect missing values. Return a boolean output
italy_companies.isnull().sum().sort_values(ascending=False).head(5)

year founded               33134
domain                     16474
industry                     597
locality                     125
total employee estimate        0
dtype: int64

In [7]:
italy_companies.head(2)

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
368,3388682,saipem,saipem.com,1957.0,oil & energy,10001+,"san donato milanese, milan, italy",italy,linkedin.com/company/saipem,14858,33895
408,5150220,unicredit,unicreditgroup.eu,2014.0,banking,10001+,"milano, lombardy, italy",italy,linkedin.com/company/unicredit,13933,25727


In [8]:
# miro los attributos localidad.La mayoria de las empresas está en Milan.
# No solo es el total de las empresas a ser mayor, si no que allí hay las empresas mas importantes,
# las que tienen un numero de empleados mayor.
# Así decido quedarme con la ciudad de Milan
italy_companies['locality'].value_counts().head(5)

milano, lombardy, italy           11699
rome, lazio, italy                 7746
torino, piedmont, italy            3495
genoa, liguria, italy              2981
bologna, emilia-romagna, italy     2754
Name: locality, dtype: int64

In [9]:
tamaño_empresas = italy_companies.sort_values(by= 'total employee estimate',ascending=False).head(100)

In [10]:
print(tamaño_empresas['locality'].str.contains(pat = 'lombard').value_counts())
print(tamaño_empresas['locality'].str.contains(pat = 'lazio').value_counts())

False    65
True     35
Name: locality, dtype: int64
False    81
True     19
Name: locality, dtype: int64


In [11]:
# Find the NaN values and change them
italy_companies['locality'] = italy_companies['locality'].fillna('No Values')
milan_companies = italy_companies[italy_companies['locality'].str.contains(pat = 'milan')]

/home/must4in3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
milan_companies.head(2)

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
368,3388682,saipem,saipem.com,1957.0,oil & energy,10001+,"san donato milanese, milan, italy",italy,linkedin.com/company/saipem,14858,33895
408,5150220,unicredit,unicreditgroup.eu,2014.0,banking,10001+,"milano, lombardy, italy",italy,linkedin.com/company/unicredit,13933,25727


In [13]:
milan_companies['industry'].value_counts().head(5)

information technology and services    1317
marketing and advertising              1285
management consulting                   865
internet                                549
design                                  400
Name: industry, dtype: int64

# 1 - Design Companies in Milan. API foursquare

In [14]:
design_companies = milan_companies[milan_companies.industry == 'design']

In [15]:
design_companies.head(2)

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
34487,4926474,naba nuova accademia di belle arti,naba.it,1980.0,design,1001 - 5000,"milan, lombardy, italy",italy,linkedin.com/company/naba-nuova-accademia-di-b...,339,969
55395,4000697,artemide,artemide.com,1960.0,design,501 - 1000,"milano, lombardy, italy",italy,linkedin.com/company/artemide,208,616


In [16]:
lugar = ['Milan']
design_category = '4bf58dd8d48988d1f4941735'
Milan_coordinate = geocode(lugar)

In [17]:
data = (getFromFoursquareCategory(design_category, 60000, Milan_coordinate))

In [18]:
design_coord = limpiaRespFoursquareCategory(data)

In [19]:
db_design = pd.DataFrame(design_coord)
#db_design = db_design[db_design.name.str.contains(pat = 'Aeroporto')]

In [20]:
db_design

,name,location
0,Spazio Rossana Orlandi,"{'address': 'Via Matteo Bandello 14-16', 'lat'..."
1,Technogym,"{'address': 'Via Durini 1', 'lat': 45.46365612..."
2,Kartell Flagship Store,"{'address': 'Via Carlo Porta, 1', 'crossStreet..."
3,10 Corso Como,"{'address': 'Corso Como 10', 'lat': 45.4818518..."
4,Echi di Carta,"{'address': 'Via Vetta d'Italia 24', 'lat': 45..."
...,...,...
60,GrioniStudio,"{'address': 'Via Hayez, 8', 'lat': 45.475299, ..."
61,Gruppo Pozzi LAB,"{'address': 'Via Amendola,8', 'lat': 45.559077..."
62,GIADA House,"{'address': 'Via Monte Napoleone 15', 'lat': 4..."
63,Como Next,{'address': 'via Camillo Benso Conte di Cavour...


In [21]:
geo_db_design = pd.DataFrame(db_design.location.apply(pd.Series))

In [22]:
# aeropuertos table limpia lista para la transformacíon en GeoPoint
clean_db_design = pd.concat([db_design["name"], geo_db_design[['address','lat','lng']]], axis=1)
clean_db_design.head(3)

,name,address,lat,lng
0,Spazio Rossana Orlandi,Via Matteo Bandello 14-16,45.463812,9.166722
1,Technogym,Via Durini 1,45.463656,9.197560
2,Kartell Flagship Store,"Via Carlo Porta, 1",45.473854,9.194827


In [23]:
clean_db_design= clean_db_design.rename(columns={"lat": "LAT_WGS84", "lng": "LONG_WGS84"})

In [24]:
clean_db_design.head(2)

,name,address,LAT_WGS84,LONG_WGS84
0,Spazio Rossana Orlandi,Via Matteo Bandello 14-16,45.463812,9.166722
1,Technogym,Via Durini 1,45.463656,9.197560


In [25]:
# para poder lanzar queries en MongoDB se necesita este formato 
# <field>: { type: "Point", coordinates: [ long, lat ] }
# por eso llamo la funcion <transformToGeoPoint as tg> y creo una nueva columna en la tabla
clean_db_design["geopoint"] = clean_db_design.apply(tg, axis=1)
clean_db_design.head(2)

,name,address,LAT_WGS84,LONG_WGS84,geopoint
0,Spazio Rossana Orlandi,Via Matteo Bandello 14-16,45.463812,9.166722,"{'type': 'Point', 'coordinates': [9.1667218642..."
1,Technogym,Via Durini 1,45.463656,9.197560,"{'type': 'Point', 'coordinates': [9.1975602304..."


In [26]:
# Export as json records to import with mongoimport 
clean_db_design.to_json("output/clean_db_design.json",orient="records")
# mongoimport -d datamad0620 -c offices --jsonArray offices.json

In [27]:
# METODOLOGIA ALTERNATIVA

In [28]:
##%%time
# de las compañias de design me falta la direccíon y la geolocalizacíon.
# a traves de un bucle for y la API foursquare hago diferentes requests por cada compañia en la tabla anterior,
# con el fin de incontrár la informacíon que necesito.
# voy a llamar la funcion <getFromFoursquare>
##design_companies_location = getFromFoursquare(design_companies)

In [29]:
# creo la nueva design_company_table, con el nombre y la geolocalizacíon
##design_company_table = pd.DataFrame(design_companies_location)

In [30]:
##design_company_table.head(2)

In [31]:
# separo la columna location en subcolumnas para recuperar latitude y longitude
# el procedimiento es necesario para luego importar los datos en MongoDB.
##geo_design_company = pd.DataFrame(design_company_table.location.apply(pd.Series))

In [32]:
# design companies table limpia lista para la transformacíon en GeoPoint
##clean_design_company = pd.concat([design_company_table["name"], geo_design_company[['address','lat','lng']]], axis=1)
##clean_design_company.head(2)

In [33]:
##clean_design_company= clean_design_company.rename(columns={"lat": "LAT_WGS84", "lng": "LONG_WGS84"})
##clean_design_company.head(2)

In [34]:
# para poder lanzar queries en MongoDB se necesita este formato 
# <field>: { type: "Point", coordinates: [ long, lat ] }
# por eso llamo la funcion <transformToGeoPoint as tg> y creo una nueva columna en la tabla
##clean_design_company["geopoint"] = clean_design_company.apply(tg, axis=1)
##clean_design_company.head(2)

In [35]:
# Export as json records to import with mongoimport 
##clean_design_company.to_json("output/design_company.json",orient="records")
# mongoimport -d datamad0620 -c offices --jsonArray offices.json

# 2- Starbucks from Selenium

In [36]:
# https://www.starbucks.it/store-locator/search/location/milano
# Web Scraping utilizando Selenium porque el solo BeautifulSoup no permite cargar toda la pagina.
# Voy a llamar la funcion <getPage> para enviar la requests a la pagina oficíal de Starbucks italia
soup = getPage("https://www.starbucks.it/store-locator/search/location/milano")

[WDM] - Current google-chrome version is 81.0.4044
[WDM] - Get LATEST driver version for 81.0.4044


[WDM] - Get LATEST driver version for 81.0.4044
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/81.0.4044.138/chromedriver_linux64.zip
[WDM] - Driver has been saved in cache [/home/must4in3/.wdm/drivers/chromedriver/linux64/81.0.4044.138]


In [37]:
# utilizando los selectores consiguo sacar el nombre de cada Starbucks y la direccíon, y el zip me permite juntar
# las dos informacíones
name_starbucks = [x.text for x in soup.select(".store-name")]
address_starbucks = [' '.join(x.text.split()) for x in soup.select(".address")]
merge_starbucks = list(zip(name_starbucks, address_starbucks))

In [38]:
# con un bucle for creo el dictionario y la tabla de los Starbucks de Milano
dict_starbucks = []
for starbucks in merge_starbucks:
    dict_starbucks.append({'name': starbucks[0], 'location': starbucks[1]})
starbucks_table = pd.DataFrame(dict_starbucks)

In [39]:
starbucks_table.head(2)

,name,location
0,Milan Roastery,Piazza Cordusio 3 Milan 20123
1,Starbucks Porta Romana,"Via Lentasio, 1 angolo Corso di Porta Romana M..."


In [40]:
%%time
# llamo la funcion geocode porque necesito añadir en la tabla las infos de lat y long
#documentación en los siguientes enlaces -> https://geojson.io/#map=16/40.3959/-3.7039 ; https://geojson.org/
starbucks_geo = geocode(starbucks_table['location'])

CPU times: user 125 ms, sys: 447 µs, total: 126 ms
Wall time: 16.9 s


In [41]:
print(starbucks_geo)

[{'type': 'Point', 'coordinates': [9.1862, 45.46496]}, {'type': 'Point', 'coordinates': [9.19167, 45.45844]}, {'type': 'Point', 'coordinates': [9.19836, 45.46563]}, None, {'type': 'Point', 'coordinates': [9.18336, 45.47405]}, {'type': 'Point', 'coordinates': [9.16487, 45.46626]}, {'type': 'Point', 'coordinates': [9.20784, 45.48631]}, {'type': 'Point', 'coordinates': [9.15274, 45.40453]}, {'type': 'Point', 'coordinates': [9.11615, 45.52215]}]


In [42]:
# con assign cada valor de la lista se va asignando a la nueva columna geopoint en la starbucks_table
starbucks_table = starbucks_table.assign(geopoint=starbucks_geo)
starbucks_table

,name,location,geopoint
0,Milan Roastery,Piazza Cordusio 3 Milan 20123,"{'type': 'Point', 'coordinates': [9.1862, 45.4..."
1,Starbucks Porta Romana,"Via Lentasio, 1 angolo Corso di Porta Romana M...","{'type': 'Point', 'coordinates': [9.19167, 45...."
2,Starbucks Durini,"Via Durini, 28 Milan 20121","{'type': 'Point', 'coordinates': [9.19836, 45...."
3,Starbucks Turati,"Via Filippo Turati, 25 Milano 20121",None
4,Starbucks Garibaldi,"Corso Garibaldi, 118 Milano 20121","{'type': 'Point', 'coordinates': [9.18336, 45...."
5,Starbucks Vercelli,"Piazzale Francesco Baracca, 10 angolo Corso Ve...","{'type': 'Point', 'coordinates': [9.16487, 45...."
6,Starbucks Centrale,"Piazza Luigi di Savoia, 1 Milano 20124","{'type': 'Point', 'coordinates': [9.20784, 45...."
7,Starbucks Assago Milanofiori,Viale Milanofiori Centro Commerciale Milanofio...,"{'type': 'Point', 'coordinates': [9.15274, 45...."
8,Starbucks Malpensa,Aeroporto Milano Malpensa T1 - Partenze Ferno ...,"{'type': 'Point', 'coordinates': [9.11615, 45...."


In [43]:
# Export as json records to import with mongoimport 
starbucks_table.to_json("output/starbucks_table.json",orient="records")
# mongoimport -d datamad0620 -c offices --jsonArray offices.json

In [44]:
# METODOLOGIA ALTERNATIVA

In [45]:
#client_id = os.getenv("CLIENT_ID")
#client_secret = os.getenv("CLIENT_SECRET")

In [46]:
## enviar una request. Si sale un numero que empieza con 2 todo bien, 
## o si no ver los errores de los gatos a ver que dicen!
#url = f'https://api.foursquare.com/v2/venues/search?near=milan&query=Starbucks&client_id={client_id}&client_secret={client_secret}&v=20181101'
#res = requests.get(url)
#starbucks= res.json()
## del requests seleciono solo la key venues dentro response
#starbucks['response']['venues']
## leer infos read_json leer documentacíon

In [47]:
## dataframe de los starbucks in Milan
#table_starbucks = pd.DataFrame((starbucks['response']['venues'])) #read json
#table_starbucks.head(3)

In [48]:
## elimino la unica row que no tenga Starbucks
#table_starbucks = table_starbucks[table_starbucks['name'].str.contains(pat = 'Starbucks')]
## separo la columna location en subcolumnas para recuperar latitude y longitude
#geo_starbucks = pd.DataFrame(table_starbucks.location.apply(pd.Series))

In [49]:
## starbucks table limpia lista para la transformacíon en GeoPoint
#clean_starbucks = pd.concat([table_starbucks["name"], geo_starbucks[['address','lat','lng']]], axis=1)
#clean_starbucks

In [50]:
## para poder hacer query en MongoDB se necesita este formato 
## <field>: { type: <GeoJSON type> , coordinates: <coordinates> }
## por eso llamo la funcion transformToGeoPoint
#clean_starbucks["geopoint"] = clean_starbucks.apply(transformToGeoPoint, axis=1)
#clean_starbucks

In [51]:
## Convert objectIds to string to be able to export as JSON 
#clean_starbucks["geopoint"] = clean_starbucks["geopoint"].apply(lambda e: str(e))
## Export as json records to import with mongoimport 
#clean_starbucks.to_json("output/starbucks.json",orient="records")
## mongoimport -d datamad0620 -c offices --jsonArray offices.json

# 3 - Sitios para diversión de los empleados

In [52]:
#https://dati.comune.milano.it/dataset/ds252-economia-locali-pubblico-spettacolo

In [53]:
dbName = "Italian_Companies"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [54]:
cur = db.Italian_Companies.find({"insegna":{'$ne':None}},{"insegna":1,"_id":0, "LAT_WGS84":1, "LONG_WGS84":1})

In [55]:
data = list(cur)

In [56]:
sitios_para_diversion = pd.DataFrame(data)
sitios_para_diversion.head(2)

,insegna,LONG_WGS84,LAT_WGS84
0,PERSEFONE,9.17573102533,45.4966580754
1,MAF CAFFE',9.18734461471,45.4815798192


In [57]:
sitios_para_diversion = sitios_para_diversion[sitios_para_diversion.LONG_WGS84 != 'null']

In [58]:
sitios_para_diversion[['LONG_WGS84', 'LAT_WGS84']] = sitios_para_diversion[['LONG_WGS84', 'LAT_WGS84']].astype(float)

In [59]:
## para poder hacer query en MongoDB se necesita este formato 
## <field>: { type: <GeoJSON type> , coordinates: <coordinates> }
## por eso llamo la funcion transformToGeoPoint
sitios_para_diversion["geopoint"] = sitios_para_diversion.apply(tg, axis=1)

In [60]:
sitios_para_diversion.head(300)

,insegna,LONG_WGS84,LAT_WGS84,geopoint
0,PERSEFONE,9.175731,45.496658,"{'type': 'Point', 'coordinates': [9.1757310253..."
1,MAF CAFFE',9.187345,45.481580,"{'type': 'Point', 'coordinates': [9.1873446147..."
2,NEW ART CAFE',9.187446,45.472379,"{'type': 'Point', 'coordinates': [9.1874460343..."
3,BANA,9.247868,45.452775,"{'type': 'Point', 'coordinates': [9.2478678237..."
4,NOY CONCEPT RESTAURANT CAFE',9.162191,45.465265,"{'type': 'Point', 'coordinates': [9.1621909244..."
...,...,...,...,...
302,FOCA LOCA,9.191531,45.461268,"{'type': 'Point', 'coordinates': [9.1915307993..."
303,BULLONA,9.160757,45.486127,"{'type': 'Point', 'coordinates': [9.1607571507..."
304,GOLD PRIVE',9.182673,45.460799,"{'type': 'Point', 'coordinates': [9.1826732674..."
305,ROCK 'N ROLL,9.207822,45.492445,"{'type': 'Point', 'coordinates': [9.2078224018..."


In [61]:
# Export as json records to import with mongoimport 
sitios_para_diversion.to_json("output/sitios_para_diversion.json",orient="records")
# mongoimport -d datamad0620 -c offices --jsonArray offices.json

# 4 - Los aeropuertos en Milan

In [62]:
# en Milan hay 2 aeropurtos principales, Milano Malpensa y Milano Linate. El tercero está situado en Bergamo,
# 60 km fuera de Milan, y el ultimo, mucho mas pequeño, llamado aeropuerto di Bresso.  

In [63]:
lugar = ['Milan']
airport_category = '4bf58dd8d48988d1ed931735'
Milan_coordinate = geocode(lugar)

In [64]:
data = (getFromFoursquareCategory(airport_category, 60000, Milan_coordinate))

In [65]:
aeropuertos_coord = limpiaRespFoursquareCategory(data)

In [66]:
db_aeropuertos = pd.DataFrame(aeropuertos_coord)
db_aeropuertos = db_aeropuertos[db_aeropuertos.name.str.contains(pat = 'Aeroporto')]

In [67]:
db_aeropuertos.head(2)

,name,location
0,Aeroporto di Milano Linate (LIN) (Aeroporto di...,"{'address': 'Viale Enrico Forlanini', 'lat': 4..."
1,Aeroporto di Bresso (LIMB),"{'address': 'Via Antonio Gramsci 1', 'lat': 45..."


In [68]:
geo_db_aeropuertos = pd.DataFrame(db_aeropuertos.location.apply(pd.Series))

In [69]:
# aeropuertos table limpia lista para la transformacíon en GeoPoint
clean_db_aeropuertos = pd.concat([db_aeropuertos["name"], geo_db_aeropuertos[['address','lat','lng']]], axis=1)
clean_db_aeropuertos.head(3)

,name,address,lat,lng
0,Aeroporto di Milano Linate (LIN) (Aeroporto di...,Viale Enrico Forlanini,45.460409,9.279444
1,Aeroporto di Bresso (LIMB),Via Antonio Gramsci 1,45.537930,9.198891
2,Aeroporto di Milano Malpensa (MXP) (Aeroporto ...,Superstrada Malpensa 2000,45.629810,8.717244


In [70]:
clean_db_aeropuertos= clean_db_aeropuertos.rename(columns={"lat": "LAT_WGS84", "lng": "LONG_WGS84"})

In [71]:
clean_db_aeropuertos.head(2)

,name,address,LAT_WGS84,LONG_WGS84
0,Aeroporto di Milano Linate (LIN) (Aeroporto di...,Viale Enrico Forlanini,45.460409,9.279444
1,Aeroporto di Bresso (LIMB),Via Antonio Gramsci 1,45.537930,9.198891


In [72]:
# para poder lanzar queries en MongoDB se necesita este formato 
# <field>: { type: "Point", coordinates: [ long, lat ] }
# por eso llamo la funcion <transformToGeoPoint as tg> y creo una nueva columna en la tabla
clean_db_aeropuertos["geopoint"] = clean_db_aeropuertos.apply(tg, axis=1)
clean_db_aeropuertos.head(2)

,name,address,LAT_WGS84,LONG_WGS84,geopoint
0,Aeroporto di Milano Linate (LIN) (Aeroporto di...,Viale Enrico Forlanini,45.460409,9.279444,"{'type': 'Point', 'coordinates': [9.2794441997..."
1,Aeroporto di Bresso (LIMB),Via Antonio Gramsci 1,45.537930,9.198891,"{'type': 'Point', 'coordinates': [9.1988912729..."


In [73]:
# Export as json records to import with mongoimport 
clean_db_aeropuertos.to_json("output/clean_db_aeropuertos.json",orient="records")
# mongoimport -d datamad0620 -c offices --jsonArray offices.json

# 5 - Basketball stadium

In [74]:
lugar = ['Milan']
basketball_category = '4bf58dd8d48988d18b941735'

In [75]:
data = (getFromFoursquareCategory(basketball_category, 60000, Milan_coordinate))

In [76]:
basketball_coord = limpiaRespFoursquareCategory(data)

In [77]:
db_basketballs = pd.DataFrame(basketball_coord)
db_basketballs.head(2)

,name,location
0,PalaCandy,"{'address': 'Viale Stucchi 16', 'lat': 45.5853..."
1,Knights Palace,"{'address': 'Via Parma 7', 'lat': 45.581818528..."


In [78]:
geo_db_basketball = pd.DataFrame(db_basketballs.location.apply(pd.Series))

In [79]:
# basketballs table limpia lista para la transformacíon en GeoPoint
clean_db_basketballs = pd.concat([db_basketballs["name"], geo_db_basketball[['address','lat','lng']]], axis=1)
clean_db_basketballs.head(2)

,name,address,lat,lng
0,PalaCandy,Viale Stucchi 16,45.585332,9.309250
1,Knights Palace,Via Parma 7,45.581819,8.894455


In [80]:
clean_db_basketballs= clean_db_basketballs.rename(columns={"lat": "LAT_WGS84", "lng": "LONG_WGS84"})

In [81]:
clean_db_basketballs.head(2)

,name,address,LAT_WGS84,LONG_WGS84
0,PalaCandy,Viale Stucchi 16,45.585332,9.309250
1,Knights Palace,Via Parma 7,45.581819,8.894455


In [82]:
# para poder lanzar queries en MongoDB se necesita este formato 
# <field>: { type: "Point", coordinates: [ long, lat ] }
# por eso llamo la funcion <transformToGeoPoint as tg> y creo una nueva columna en la tabla
clean_db_basketballs["geopoint"] = clean_db_basketballs.apply(tg, axis=1)
clean_db_basketballs.head(2)

,name,address,LAT_WGS84,LONG_WGS84,geopoint
0,PalaCandy,Viale Stucchi 16,45.585332,9.309250,"{'type': 'Point', 'coordinates': [9.3092497824..."
1,Knights Palace,Via Parma 7,45.581819,8.894455,"{'type': 'Point', 'coordinates': [8.8944551988..."


In [83]:
# Export as json records to import with mongoimport 
clean_db_basketballs.to_json("output/clean_db_basketballs.json",orient="records")
# mongoimport -d datamad0620 -c offices --jsonArray offices.json

# Numeros civicos con coordenadas de toda Milan
https://dati.comune.milano.it/dataset/ds634-numeri-civici-coordinate

In [84]:
# en la página web puedo descargar el json de todas las direcciónes, por cada piso de la ciudad, georreferenciadas.
dbName = "Italian_Companies"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [85]:
cur = db.civici_coordinate.find({"DESCRITTIVO":{'$ne':None}},{"_id":0, "TIPO":1, "DESCRITTIVO":1, "NUMEROCOMPLETO":1,"LAT_WGS84":1, "LONG_WGS84":1})

In [86]:
data = list(cur)

In [87]:
n_civicos_milan = pd.DataFrame(data) 

In [88]:
n_civicos_milan.head(2)

,NUMEROCOMPLETO,TIPO,DESCRITTIVO,LONG_WGS84,LAT_WGS84
0,21N08,Piazza,DEL DUOMO,9.189762,45.464840
1,21N01,Piazza,DEL DUOMO,9.189314,45.464846


In [89]:
n_civicos_milan["geopoint"] = n_civicos_milan.apply(tg, axis=1)

In [90]:
n_civicos_milan.head(2)

,NUMEROCOMPLETO,TIPO,DESCRITTIVO,LONG_WGS84,LAT_WGS84,geopoint
0,21N08,Piazza,DEL DUOMO,9.189762,45.464840,"{'type': 'Point', 'coordinates': [9.189761993,..."
1,21N01,Piazza,DEL DUOMO,9.189314,45.464846,"{'type': 'Point', 'coordinates': [9.1893140941..."


In [91]:
# Export as json records to import with mongoimport 
n_civicos_milan.to_json("output/clean_civicos_milan.json",orient="records")
# en la terminal para utilizar el json en mongodb
# mongoimport --db Italian_Companies --collection clean_civicos_milan --file clean_civicos_milan.json --jsonArray